## The Question

An important question in criminal justice is whether detaining defendants while they await trial affects their ultimate probability of conviction. In this final exam you will use machine learning to estimate the causal effect of pre-trial detention on conviction. In the [shared Econ 484 Google Drive "datasets" folder](https://drive.google.com/drive/folders/1gAPP5E0zb4XRd8Ey9eyJ48IIlCCutU9c?usp=sharing) you will find a dataset called "miami_example.csv" and the associated codebook "miami_defs.txt" that gives some information about each of the variables. The outcome variable is the indicator for conviction. The treatment variable is an indicator pre-trial detention. Possible instrumental variables are dummy variables for each judge. You will have to create the dummy variables from the judge id variable. Additional covariates are age, gender, race, prior offender indicator, number of counts, and indicators for type of crimes in the current charge.

## The Task

Estimate the causal effect of pre-trial detention on conviction in two ways:

1) Via OLS regression where you use machine learning to control for the additional covariates.

2) Via instrumental variables regression using dummy variables for bail judge  as instruments for pre-trial detention where you use machine learning to address the problems that can arise with many instruments (there are 166 judges). 

In [ ]:
# mount drive for access to data
from google.colab import drive
drive.mount('/content/drive/')

# import necessary packages
import pandas as pd
from numpy import mean, ravel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression
from sklearn.neural_network import MLPRegressor

# load in data, assuming NaNs are random
dataset = pd.read_csv('/content/drive/MyDrive/Econ 484/miami_example.csv')
dataset = dataset.dropna()

# generate y (outcome), d (treatment), x (controls) arrays
y = dataset[['any_guilty']]
d = dataset[['detained']]
x = dataset.drop(['any_guilty', 'detained'], axis=1)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# 1) Via OLS regression with ML to control additional covariates
# METHOD: DDML (using Ridge) with sample splitting

# cross validate to find best parameters
y_alpha = RidgeCV().fit(x, y).alpha_
d_alpha = RidgeCV().fit(x, d).alpha_

# Sample splitting: predict residuals for y, d using x for each split
y_hats = []
d_hats = []
kf = KFold(2).split(dataset)
for train, test in kf:
  y_hat = Ridge(y_alpha).fit(x.iloc[train], y.iloc[train]).predict(x.iloc[test])
  y_hats.append(y_hat)

  d_hat = Ridge(d_alpha).fit(x.iloc[train], d.iloc[train]).predict(x.iloc[test])
  d_hats.append(d_hat)

# generate ytilde & dtilde by subtracting off residual's mean
ytilde = y - mean(y_hats)
dtilde = d - mean(d_hats)

# estimate the causal effect using OLS regression
ddml_beta = LinearRegression().fit(dtilde, ytilde).coef_

print('The DDML-estimated causal effect of pre-trial detention on conviction is {:.4}'.format(ddml_beta.item()))

The DDML-estimated causal effect of pre-trial detention on conviction is 0.2586


In [ ]:
# 2) Via IV with ML to control for many instruments
# METHOD: ML-augmented instrumental variables

# create instruments: dummy variables for judgeid
judgeid = pd.get_dummies(x['judgeid'])

# cross validate to find best parameters
net = MLPRegressor(max_iter=30)
param_grid = {'hidden_layer_sizes':[(2,), (10,), (25,)], 
              'activation':['relu', 'logistic']}
gridsearch = GridSearchCV(net, param_grid)

# predict d using net with best parameters
d_hat = gridsearch.fit(judgeid, ravel(d)).predict(judgeid).reshape(-1,1)

# estimate the causal effect using 2SLS IV with d_hat as single instrument
firststage = LinearRegression().fit(d_hat, d).predict(d_hat).reshape(-1,1)
secondstage = LinearRegression().fit(firststage, y)
iv_beta = secondstage.coef_

print('The ML-augmented IV causal effect of pre-trial detention on conviction is {:.4}'.format(iv_beta.item()))

The ML-augmented IV causal effect of pre-trial detention on conviction is 1.136
